# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [407]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=0"

In [408]:
import requests
import bs4
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import numpy as np

In [409]:
## YOUR CODE HERE

page = urllib2.urlopen(URL)

#parse the page
soup = BeautifulSoup(requests.get(URL).text)

#extracting all results
result = soup.find("div", {"class": "row result"},'lxml')


Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

## YOUR CODE HERE
def extract_location_from_result(result):
    for loc in soup.findAll('span', {'class':'location'}):
        return loc.get_text(strip=True)

def extract_company_from_result(result):
    for loc in soup.findAll('span', {'class':'company'}):
        return loc.get_text(strip=True)

salary = []

def extract_salary_from_result(result):
    for loc in soup.findAll('span', {'class': 'location'}):
        Sibling = soup.findNextSiblings('div')
        if Sibling != None:
            salary.append(loc.text.strip())
        else:
            salary.append(np.nan)

def extract_title_from_result(result):
    for loc in soup.findAll('a', {'data-tn-element':'jobTitle'}):
        return loc['title']
    
#print extract_title_from_result(result)


In [410]:
#def extract_text(el):
#    if el:
#        return el.text.strip()
#    else:
#        return ''
        
def extract_company_from_result(result):
    return result.find('span', {'class' : 'company'}).text.strip()

def extract_location_from_result(result):
    return  result.find('span', {'class' : 'location'}).text.strip()

#def extract_summary_from_result(result):
#    return  extract_text(result.find('span', {'class' : 'summary'}))

def extract_title_from_result(result):
    return result.find('a', {'data-tn-element' : 'jobTitle'}).text.strip()

def extract_salary_from_result(result):
    salary_table = result.find('td', {'class' : 'snip'})
    if salary_table:
        snip = salary_table.find('nobr')
        if snip:
            return snip.text.strip()
        else:
            return np.nan
        
#print extract_salary_from_result(result)

In [411]:
lst1 = [loc.get_text(strip=True) for loc in soup.findAll('span', {'class':'company'})]
print (lst1)

lst2 = [loc.get_text(strip=True) for loc in soup.findAll('span', {'class':'location'})]
print (lst2)

lst3 = [loc.get_text(strip=True) for loc in soup.findAll('a', {'data-tn-element':'jobTitle'})]#dual conditions?
print (lst3)

#lst3 = [loc.findNextSibling('div') for loc in soup.findAll('div', {'class':'sjcl'})]
#print (lst3)


[u'Cipla', u'1010data', u'New York University College of Dentistry', u'Globant', u'NBCUniversal', u'Memorial Sloan Kettering Cancer Center', u'Bloomberg', u'Columbia University', u'eTemp', u'NBCUniversal', u'CAPCO', u'Celmatix', u'S&P Global Ratings', u'Uber', u'AbilTo, Inc']
[u'Hauppauge, NY', u'New York, NY', u'New York, NY', u'New York, NY', u'New York, NY', u'New York, NY 10065(Upper East Side area)', u'New York, NY', u'New York, NY', u'New York, NY', u'New York, NY', u'New York, NY 10271(Financial District area)', u'New York, NY 10005(Financial District area)', u'New York, NY 10002(Lower East Side area)', u'New York, NY', u'New York, NY']
[u'SeniorScientist', u'VP of Product,DataInsights', u'JUNIOR RESEARCHSCIENTISTMcDevitt Research Laboratory', u'SsrDataScientist(R, Tableau, Python)', u'DataEngineer, Analytics Technology', u'BioinformaticsDataScientist- Cancer Genomics, NYC', u'Machine Learning Engineer', u'Associate ResearchScientist', u'Mid - Senior LevelDataScientist', u'Resea

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [412]:
YOUR_CITY = 'Atlanta'

In [413]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%24{}%2C000&l={}&start={}"
max_results_per_city = 500

rows = []

#look through each cities
for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Los+Angeles',YOUR_CITY,'Seattle']):
    #look through pages
    for start in range(0, max_results_per_city, 10):
        #parse the page
        #If the salary is set to 0, the link will search for just the term data scientist
        #If the salary is set to 100, the link will search for data scientist with $100k+ salary
        for salary in set([0,100]):
            url = url_template.format(salary,city,start)
            soup = BeautifulSoup(requests.get(url).text)
            #extracting all results
            results = soup.findAll("div", {"class": "row result"})
            #search data scienctist without any specific salary
            if salary == 0:
                for result in results:
                    if result:
                        row = {}
                        row['title'] =  extract_title_from_result(result)
                        row['company'] = extract_company_from_result(result)
                        row['salary'] = 0 #add 0s for salary under 100k
                        row['city'] = city
                        rows.append(row)
            #search data scientist with $100k salary       
            else:
                for result in results:
                    if result:
                        row = {}
                        row['title'] =  extract_title_from_result(result)
                        row['company'] = extract_company_from_result(result)
                        row['salary'] = 1 #add 1s for salary 100k and over
                        row['city'] = city
                        rows.append(row)
                    

In [414]:
rows

[{'city': 'Chicago',
  'company': u'Gallup',
  'salary': 0,
  'title': u'Data Scientist \u2013 Predictive Analytics'},
 {'city': 'Chicago',
  'company': u'Parexel International',
  'salary': 0,
  'title': u'Senior Statistical Programmer (Home Based)'},
 {'city': 'Chicago',
  'company': u'Parallel Partners',
  'salary': 0,
  'title': u'Junior Quantitative Research Analyst - MS062161'},
 {'city': 'Chicago',
  'company': u'Gallup',
  'salary': 1,
  'title': u'Data Scientist \u2013 Predictive Analytics'},
 {'city': 'Chicago',
  'company': u'Next Step Systems',
  'salary': 1,
  'title': u'Junior Quantitative Research Analyst'},
 {'city': 'Chicago',
  'company': u'Allstate Insurance',
  'salary': 1,
  'title': u'Data Science Manager'},
 {'city': 'Chicago',
  'company': u'CVS Health',
  'salary': 0,
  'title': u'Predictive Modeling, Strategic Analytics'},
 {'city': 'Chicago',
  'company': u'Parexel International',
  'salary': 0,
  'title': u'Senior Statistical Programmer (Home Based)'},
 {'ci

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [415]:
## YOUR CODE HERE

#create a dataframe
data = pd.DataFrame.from_records(rows)


#data.describe()
#data.sort_values(by='salary')

#drop duplicates but keep the last. Since there was a duplicate result for a search with the salary requirement of 100k
#one without, drop the first one and keep the last search which is for 100k.
data = data.drop_duplicates(['city','company','title'],keep='last')
data.head(20)

,city,company,salary,title
154,Chicago,Cameron Craig Group,1,Application Scientist
179,Chicago,Next Step Systems,1,Junior Quantitative Research Analyst
227,Chicago,Allstate Insurance,1,Senior Data Scientist
259,Chicago,Thermo Fisher Scientific,0,Applications Scientist
260,Chicago,Parexel International,0,"Associate Manager, Statistical Programming"
282,Chicago,Parexel International,0,Senior Statistical Programmer (Home Based)
286,Chicago,Parallel Partners,1,Junior Quantitative Research Analyst - MS062161
289,Chicago,Allstate Insurance,0,Machine Learning Engineer - Connected Car
295,Chicago,CVS Health,0,"Predictive Modeling, Strategic Analytics"
296,Chicago,Allstate Insurance,0,Data Science Manager


Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [416]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [417]:
## YOUR CODE HERE

### Save your results as a CSV

In [418]:
## YOUR CODE HERE

datadata = data.to_csv('../assets/cleanIndeed1.csv',encoding='utf-8')

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [419]:
## YOUR CODE HERE

data = pd.DataFrame.from_csv('../assets/cleanIndeed.csv')

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [433]:
## YOUR CODE HERE

,city,company,salary,title
344,False,False,False,False
345,False,False,False,False
348,False,False,False,False
349,False,False,False,False
350,False,False,False,False
351,False,False,False,False
352,False,False,False,False
353,False,False,False,False
354,False,False,False,False
355,False,False,False,False


#### Thought experiment: What is the baseline accuracy for this model?

In [421]:
## YOUR CODE HERE

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [422]:
#create dummy variables for cities
dummy_location = pd.get_dummies(data['city'], prefix='city')
cols_to_keep = ['company', 'salary', 'title']
handCalc = data[cols_to_keep].join(dummy_location.ix[:, :])
#print handCalc.head()
#print pd.crosstab(handCalc['salary'], handCalc['city_New+York'], rownames=['salary'])

In [423]:
## YOUR CODE HERE
import statsmodels.api as sm
cols_to_keep = ['city', 'salary', 'title']
df = data[cols_to_keep].join(dummy_location.ix[:, 'city_Austin':])
#print df.head()
df['intercept'] = 1.0
#print df.ix[:, df.columns != 'salary']

#use statsmodels to perform logistic regression
logit = sm.Logit(df['salary'], df.ix[:,3:])
result = logit.fit()

         Current function value: 0.527339
         Iterations: 35


/Users/SangYeob/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [424]:
dfNew = df

#make a column for prediction and use the model to get the probability
dfNew['prediction'] = result.predict(df.ix[:,3:])
threshold = 0.5

#make a column and use a threshold of 0.5(best threshold I could find) to predict whether it's 0 or 1.
dfNew['approxPredict'] = [0 if i<threshold else 1 for i in dfNew['prediction']]

#confusion matrix of the results
print pd.crosstab(dfNew['approxPredict'],dfNew['salary'], rownames=['predicted'])

salary      0   1
predicted        
0          16   6
1          12  27


In [425]:
#numerical values of the coefficients and how strongly they affect the salary compared to Atlanta(since Atlanta was dropped)
print np.exp(result.params)
params = result.params

#confidence interval of 2.5% and 97.5%
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

city_Austin           4.398826e+09
city_Chicago          3.299119e+09
city_Los+Angeles      1.055718e+10
city_New+York         1.099706e+10
city_San+Francisco    1.154692e+10
city_Seattle          5.042694e+20
intercept             1.515556e-10
dtype: float64
                    2.5%  97.5%            OR
city_Austin          0.0    inf  4.398826e+09
city_Chicago         0.0    inf  3.299119e+09
city_Los+Angeles     0.0    inf  1.055718e+10
city_New+York        0.0    inf  1.099706e+10
city_San+Francisco   0.0    inf  1.154692e+10
city_Seattle         0.0    inf  5.042694e+20
intercept            0.0    inf  1.515556e-10


In order to get a 6-figure salary, it's best to go to these cities with higher exp(coef) values. 

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [426]:
## YOUR CODE HERE


#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [427]:
## YOUR CODE HERE
from sklearn import linear_model

#scikit-learn logistic regression model
lm = linear_model.LogisticRegression()
#target variable/predictor is salary
y = df[['salary']]
#using all the dummy variables as predictors
X = df.ix[:,3:]

result = lm.fit(X,y)

#predicted values from the scikit-learn logistic regression model
predictions = lm.predict(X)

#plt.bar(predictions,y)
#predictions

/Users/SangYeob/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [428]:
## YOUR CODE HERE
from sklearn.cross_validation import train_test_split
y = df.salary
X = df.ix[:,3:]

#cross validation: setting up my train sets and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#creating a model using train sets
logReg = sm.Logit(y_train,X_train)
result = logReg.fit()
result.summary()

         Current function value: 0.604510
         Iterations: 35


/Users/SangYeob/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 salary   No. Observations:                   42
Model:                          Logit   Df Residuals:                       36
Method:                           MLE   Df Model:                            5
Date:                Fri, 01 Jul 2016   Pseudo R-squ.:                  0.1221
Time:                        08:43:34   Log-Likelihood:                -25.389
converged:                      False   LL-Null:                       -28.921
                                        LLR p-value:                    0.2159
======================================================================================
                         coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------
city_Austin           -2.3850        nan        nan        nan           nan       nan
city_Chicago          -2.1006   9.04e+06  -2.32e-07      1.000     -1.77e+07  1.77e+07
city_Los+Angeles      -6.3497        nan        nan        nan           nan       nan
city_New+York         -6.5376        nan        nan        nan           nan       nan
city_San+Francisco    -6.3474        nan        nan        nan           nan       nan
city_Seattle          22.2677        nan        nan        nan           nan       nan
intercept             -1.4527   3.03e+07   -4.8e-08      1.000     -5.93e+07  5.93e+07
prediction             8.5806        nan        nan        nan           nan       nan
approxPredict          3.0329   1.75e+07   1.73e-07      1.000     -3.43e+07  3.43e+07
======================================================================================
"""

In [429]:
#the actual values and the p values from the table above shows that most of my variables doesn't mean anything
print np.exp(result.params)
params = result.params
#confidence interval for the cross validation train set model
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

city_Austin           9.208961e-02
city_Chicago          1.223791e-01
city_Los+Angeles      1.747206e-03
city_New+York         1.447954e-03
city_San+Francisco    1.751244e-03
city_Seattle          4.685152e+09
intercept             2.339304e-01
prediction            5.327566e+03
approxPredict         2.075723e+01
dtype: float64
                    2.5%  97.5%            OR
city_Austin          NaN    NaN  9.208961e-02
city_Chicago         0.0    inf  1.223791e-01
city_Los+Angeles     NaN    NaN  1.747206e-03
city_New+York        NaN    NaN  1.447954e-03
city_San+Francisco   NaN    NaN  1.751244e-03
city_Seattle         NaN    NaN  4.685152e+09
intercept            0.0    inf  2.339304e-01
prediction           NaN    NaN  5.327566e+03
approxPredict        0.0    inf  2.075723e+01


In [430]:
X_test['actualSalary'] = y_test
dfTrain = X_test
dfTrain['predictedSalary'] = result.predict(dfTrain[dfTrain.columns[0:7]])
xyz = dfTrain['predictedSalary']
threshold = 0.4
dfTrain['approxPredict'] = [0 if i<threshold else 1 for i in xyz]

#confusion matrix cross validation model
print pd.crosstab(dfTrain['approxPredict'],dfTrain['actualSalary'], rownames=['predicted'])

/Users/SangYeob/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


ValueError: shapes (19,7) and (9,) not aligned: 7 (dim 1) != 9 (dim 0)

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [ ]:
## YOUR CODE HERE
#L1 = Lasso
#penalizes the coeffecients by dropping coefficients

In [ ]:
## YOUR CODE HERE
#L2 = Ridge
#penalizes the coefficients by reducing the coefficients by ^2. 

In [ ]:
from IPython.display import Image
from IPython.display import display
%matplotlib inline
#tableau picture that lists total sales by zip codes
x = Image(filename='../assets/images/average salary.png')
display(x)
#the bargraph is average of the salaries. If the number is higher, it means that those particular cities have
#more number of jobs that is in 6 figures.

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [ ]:
## YOUR CODE HERE